### Set up

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pyspark.sql.functions as F
import pyspark.sql.types as T 
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [2]:
# Create a Spark session now to access functionalities of Spark
spark = SparkSession.builder \
.config("spark.executor.memory", "4g")\
.config("spark.driver.memory", "10g")\
.master("local[6]") \
.appName("GroupProject") \
.getOrCreate()

spark

### Read data

In [3]:
file_names_range = list(range(2009, 2018))
# file_paths = [f'd://Oulu university courses//Big data processing and applications//Assignment//Data 2009-2018//{file}.csv' for file in file_names_range]
file_paths = [f'D:\Oulu university courses\Big data processing and applications\Assignment\Data 2009-2018\{file}.csv' for file in file_names_range]
print(file_paths)

['D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2009.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2010.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2011.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2012.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2013.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2014.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2015.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2016.csv', 'D:\\Oulu university courses\\Big data processing and applications\\Assignment\\Data 2009-2018\\2017.csv']


In [4]:
schema = T.StructType([
    T.StructField("FL_DATE", T.TimestampType(), nullable=True),
    T.StructField("OP_CARRIER", T.StringType(), nullable=True),
    T.StructField("OP_CARRIER_FL_NUM", T.IntegerType(), nullable=True),
    T.StructField("ORIGIN", T.StringType(), nullable=True),
    T.StructField("DEST", T.StringType(), nullable=True),
    T.StructField("CRS_DEP_TIME", T.DoubleType(), nullable=True),
    T.StructField("DEP_TIME", T.DoubleType(), nullable=True),
    T.StructField("DEP_DELAY", T.DoubleType(), nullable=True),
    T.StructField("TAXI_OUT", T.DoubleType(), nullable=True),
    T.StructField("WHEELS_OFF", T.DoubleType(), nullable=True),
    T.StructField("WHEELS_ON", T.DoubleType(), nullable=True),
    T.StructField("TAXI_IN", T.DoubleType(), nullable=True),
    T.StructField("CRS_ARR_TIME", T.DoubleType(), nullable=True),
    T.StructField("ARR_TIME",T.DoubleType(), nullable=True),
    T.StructField("ARR_DELAY", T.DoubleType(), nullable=True),
    T.StructField("CANCELLED", T.DoubleType(), nullable=True),
    T.StructField("CANCELLATION_CODE", T.StringType(), nullable=True),
    T.StructField("DIVERTED", T.DoubleType(), nullable=True),
    T.StructField("CRS_ELAPSED_TIME", T.DoubleType(), nullable=True),
    T.StructField("ACTUAL_ELAPSED_TIME", T.DoubleType(), nullable=True),
    T.StructField("AIR_TIME", T.DoubleType(), nullable=True),
    T.StructField("DISTANCE", T.DoubleType(), nullable=True),
    T.StructField("CARRIER_DELAY", T.DoubleType(), nullable=True),
    T.StructField("WEATHER_DELAY", T.DoubleType(), nullable=True),
    T.StructField("NAS_DELAY", T.DoubleType(), nullable=True),
    T.StructField("SECURITY_DELAY", T.DoubleType(), nullable=True),
    T.StructField("LATE_AIRCRAFT_DELAY", T.DoubleType(), nullable=True),
    T.StructField("Unnamed: 27", T.StringType(), nullable=True)
])

In [5]:
df = spark.read.schema(schema).format("csv").option("header", "true").load(file_paths)

### Get an overview of data

In [6]:
df.show(5)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------

In [7]:
df.columns

['FL_DATE',
 'OP_CARRIER',
 'OP_CARRIER_FL_NUM',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'Unnamed: 27']

In [8]:
df.describe().show()

+-------+----------+------------------+--------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+--------------------+-----------------+--------------------+-----------------+-------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+-----------+
|summary|OP_CARRIER| OP_CARRIER_FL_NUM|  ORIGIN|    DEST|      CRS_DEP_TIME|          DEP_TIME|         DEP_DELAY|          TAXI_OUT|        WHEELS_OFF|        WHEELS_ON|          TAXI_IN|      CRS_ARR_TIME|         ARR_TIME|        ARR_DELAY|           CANCELLED|CANCELLATION_CODE|            DIVERTED| CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|          AIR_TIME|         DISTANCE|     CARRIER_DELAY|     WEATHER_DELAY|         NAS_DELAY|     SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------+----------+----

### Clean data

In [9]:
# Remove null values from the cols used for classification and save df for analysis
analysis_df = df.dropna(subset= [
    'FL_DATE',
 'OP_CARRIER',
 'OP_CARRIER_FL_NUM',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'CRS_ARR_TIME',
 'CANCELLED',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'DISTANCE'])

In [20]:
analysis_df.count()

54343467

In [10]:
# Drop the cols which indirectly indicate if a flight is cancelled or not (apart from the column CANCELLED)
# Most of those cols contain null values, if the flight is cancelled

classify_df = df.drop("Unnamed: 27", 
                        "CARRIER_DELAY", 
                        "WEATHER_DELAY",
                        "NAS_DELAY",
                        "SECURITY_DELAY",
                        "LATE_AIRCRAFT_DELAY",
                        "CANCELLATION_CODE",
                        "DEP_TIME",
                        "DEP_DELAY",
                        "TAXI_OUT",
                        "WHEELS_OFF",
                        "WHEELS_ON",
                        "TAXI_IN",
                        "ARR_TIME",
                        "ARR_DELAY",
                        "ACTUAL_ELAPSED_TIME", 
                        "AIR_TIME")

In [11]:
# Convert timestamp column to numerical
classify_df = classify_df.withColumn("FL_DATE", F.unix_timestamp("FL_DATE"))

In [12]:
classify_df.columns

['FL_DATE',
 'OP_CARRIER',
 'OP_CARRIER_FL_NUM',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'CRS_ARR_TIME',
 'CANCELLED',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'DISTANCE']

In [13]:
classify_df.show(5)

+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+
|1230760800|        XE|             1204|   DCA| EWR|      1100.0|      1202.0|      0.0|     0.0|            62.0|   199.0|
|1230760800|        XE|             1206|   EWR| IAD|      1510.0|      1632.0|      0.0|     0.0|            82.0|   213.0|
|1230760800|        XE|             1207|   EWR| DCA|      1100.0|      1210.0|      0.0|     0.0|            70.0|   199.0|
|1230760800|        XE|             1208|   DCA| EWR|      1240.0|      1357.0|      0.0|     0.0|            77.0|   199.0|
|1230760800|        XE|             1209|   IAD| EWR|      1715.0|      1900.0|      0.0|     0.0|           105.0|   213.0|


In [14]:
classify_df.count()

54343518

In [15]:
# Take a subset: either balanced (with subsampling) or unbalanced
# we take a subset, because of memory limitations

# select subsample of positive samples - 10%
pos_df = classify_df.filter(F.col('CANCELLED').isin(1)).sample(fraction=0.1)
# select an equal amount of negative samples (number of neg samples == number of pos samples)
neg_df = classify_df.filter(F.col('CANCELLED').isin(0)).orderBy(F.rand()).limit(pos_df.count())


# Combine pos_df and neg_df - 171146 rows
classify_df = pos_df.union(neg_df).sample(fraction=1.0).cache()
classify_df.show(5)

+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+
|1230760800|        OO|             4810|   FAT| SLC|       630.0|       910.0|      1.0|     0.0|           100.0|   501.0|
|1230760800|        OO|             5748|   LAX| CLD|      1838.0|      1919.0|      1.0|     0.0|            41.0|    86.0|
|1230760800|        OO|             5750|   CLD| LAX|      1715.0|      1758.0|      1.0|     0.0|            43.0|    86.0|
|1230760800|        OO|             7782|   SUN| SLC|       938.0|      1050.0|      1.0|     0.0|            72.0|   223.0|
|1230760800|        UA|              405|   DEN| SNA|      1433.0|      1600.0|      1.0|     0.0|           147.0|   846.0|


In [16]:
pos_df.count()

85573

In [18]:
neg_df.count()

85573

In [19]:
classify_df.count()

171146

### Analyze data - using analysis_df

In [21]:
# Top 10 carriers that have the highest number of flights
carriers_flight_count_df = analysis_df.groupBy(F.col('OP_CARRIER')).count().orderBy(F.col('count').desc())
top_10 = carriers_flight_count_df.limit(10).toPandas()
top_10 = top_10.rename(columns={'OP_CARRIER':'Carrier'})
top_10

,Carrier,count
0,WN,10743987
1,DL,6892597
2,AA,5765343
3,OO,5488915
4,EV,4523555
5,UA,4205093
6,MQ,2920259
7,US,2657286
8,B6,2178956
9,AS,1414778


In [22]:
import altair as alt

# Visualisation of top 10 carriers that have the highest number of flights
chart = alt.Chart(top_10).mark_arc(outerRadius=260, innerRadius=75).encode(
    theta = alt.Theta(field="count", type="quantitative", stack=True),
    color = alt.Color('Carrier:N', scale=alt.Scale(scheme='category20'), legend=None),
).properties(
    title='Top 10 Carriers by number of flights',
    width=600,
    height=300
)

pie = chart.mark_arc(outerRadius=350)
value_text = pie.mark_text(radius=300, size=15).encode(text=alt.Text('count:Q'))

pie2 = chart.mark_arc(outerRadius=250)
text = pie2.mark_text(radius=200, size=15).encode(
    text=alt.Text('Carrier:N'), 
    color=alt.value("#000000")
)

(chart + text + value_text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=18
)

alt.LayerChart(...)

In [28]:
# Count number of cancellations per code/reason
carriers_flight_count_df = analysis_df.filter(F.col('CANCELLATION_CODE').isNotNull()).groupBy(F.col('CANCELLATION_CODE')).count()
cancellation_reasons = carriers_flight_count_df.toPandas()
cancellation_reasons

,CANCELLATION_CODE,count
0,B,414449
1,C,155532
2,A,285946
3,D,664


In [30]:
# Interpret cancellation codes to reasons
cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'A'] = 'By carrier'
cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'B'] = 'Due to weather'
cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'C'] = 'By national air system'
cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'D'] = 'For security'
cancellation_reasons = cancellation_reasons.rename(columns={'CANCELLATION_CODE':'Reason'})
cancellation_reasons

C:\Users\Pham Phuoc\AppData\Local\Temp\ipykernel_23816\1043396036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'A'] = 'By carrier'
C:\Users\Pham Phuoc\AppData\Local\Temp\ipykernel_23816\1043396036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cancellation_reasons['CANCELLATION_CODE'][cancellation_reasons['CANCELLATION_CODE'] == 'B'] = 'Due to weather'
C:\Users\Pham Phuoc\AppData\Local\Temp\ipykernel_23816\1043396036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

,Reason,count
0,Due to weather,414449
1,By national air system,155532
2,By carrier,285946
3,For security,664


In [31]:
# Visualisation of calcellation reasons
chart = alt.Chart(cancellation_reasons).mark_arc(outerRadius=180, innerRadius=50).encode(
    theta = alt.Theta(field="count", type="quantitative", stack=True),
    color = alt.Color('Reason:N', scale=alt.Scale(scheme='category20'), legend=None),
).properties(
    title='Reasons for flight cancellations',
    width=600,
    height=300
)

pie = chart.mark_arc(outerRadius=250)
value_text = pie.mark_text(radius=220, size=15).encode(text=alt.Text('count:Q'))

pie2 = chart.mark_arc(outerRadius=150)
text = pie2.mark_text(radius=120, size=12).encode(
    text=alt.Text('Reason:N'), 
    color=alt.value("#000000")
)

(chart + text + value_text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=18
)

alt.LayerChart(...)

In [47]:
# Get flights that are delayed (have departure delay time > 0)
departure_delay_df = analysis_df.filter(F.col('DEP_DELAY') > 0).select('OP_CARRIER' , 'OP_CARRIER_FL_NUM', 'DEP_DELAY').orderBy(F.col('DEP_DELAY').desc())

In [48]:
# Top 10 flights with longest departure delay time (in minutes)
top_10_delay_flights = departure_delay_df.limit(10).toPandas()
top_10_delay_flights

,OP_CARRIER,OP_CARRIER_FL_NUM,DEP_DELAY
0,AA,611,2755.0
1,NW,598,2445.0
2,AA,1389,2402.0
3,AA,239,2208.0
4,AA,1164,2149.0
5,AA,1164,2040.0
6,AA,1242,2032.0
7,AA,1322,1988.0
8,AA,1202,1975.0
9,AA,2604,1970.0


In [49]:
# Top 10 carriers that have the highest number of delay flights 
top_10_delay_carriers = departure_delay_df.groupBy(F.col('OP_CARRIER')).count().orderBy(F.col('count').desc())
top_10_delay_carriers = top_10_delay_carriers.limit(10).toPandas()
top_10_delay_carriers = top_10_delay_carriers.rename(columns={'OP_CARRIER':'Carrier'})
top_10_delay_carriers

,Carrier,count
0,WN,5417483
1,DL,2212883
2,AA,2066426
3,UA,1723634
4,OO,1613490
5,EV,1511385
6,MQ,957968
7,B6,814978
8,US,754703
9,FL,376721


In [50]:
import altair as alt

# Visualisation of top 10 carriers that have the highest number of delay flights 
chart = alt.Chart(top_10_delay_carriers).mark_arc(outerRadius=260, innerRadius=75).encode(
    theta = alt.Theta(field="count", type="quantitative", stack=True),
    color = alt.Color('Carrier:N', scale=alt.Scale(scheme='category20'), legend=None),
).properties(
    title='Top 10 Carriers by number of delay flights',
    width=600,
    height=300
)

pie = chart.mark_arc(outerRadius=350)
value_text = pie.mark_text(radius=300, size=15).encode(text=alt.Text('count:Q'))

pie2 = chart.mark_arc(outerRadius=250)
text = pie2.mark_text(radius=200, size=15).encode(
    text=alt.Text('Carrier:N'), 
    color=alt.value("#000000")
)

(chart + text + value_text).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=18
)

alt.LayerChart(...)

### Preprocess data for training models

In [51]:
# Define StringIndexer: categorical (string) cols -> to column indices, 
# Each category gets a integer based on their frequency (start from 0)

carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_Index")
origin_indexer = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_Index")
dest_indexer = StringIndexer(inputCol="DEST", outputCol="DEST_Index")

In [52]:
# Define onehotencoder for a index columns 
onehotencoder_carrier_vector = OneHotEncoder(inputCol="OP_CARRIER_Index", outputCol="OP_CARRIER_vec")
onehotencoder_origin_vector = OneHotEncoder(inputCol="ORIGIN_Index", outputCol="ORIGIN_vec")
onehotencoder_dest_vector = OneHotEncoder(inputCol="DEST_Index", outputCol="DEST_vec")

In [53]:
# Pipelining the preprocessing stages defined above 
pipeline = Pipeline(stages=[carrier_indexer, origin_indexer, dest_indexer,
                            onehotencoder_carrier_vector, onehotencoder_origin_vector,
                            onehotencoder_dest_vector])

transformed_df = pipeline.fit(classify_df).transform(classify_df)

In [57]:
transformed_df.show(5)

+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+----------------+------------+----------+--------------+-----------------+-----------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|DISTANCE|OP_CARRIER_Index|ORIGIN_Index|DEST_Index|OP_CARRIER_vec|       ORIGIN_vec|         DEST_vec|
+----------+----------+-----------------+------+----+------------+------------+---------+--------+----------------+--------+----------------+------------+----------+--------------+-----------------+-----------------+
|1230760800|        OO|             4810|   FAT| SLC|       630.0|       910.0|      1.0|     0.0|           100.0|   501.0|             2.0|        99.0|      19.0|(20,[2],[1.0])| (343,[99],[1.0])| (339,[19],[1.0])|
|1230760800|        OO|             5748|   LAX| CLD|      1838.0|      1919.0|      1.0|     0.0|            41.0|    86.0|        

In [54]:
# Select columns that are combined to one feature column
feature_columns = transformed_df.columns

# Remove cols that whould not be in our feature cols (label col, intermediate preprocessing cols)
for item in ["CANCELLED", "ORIGIN", "DEST", "OP_CARRIER", "OP_CARRIER_Index", "ORIGIN_Index", "DEST_Index"]:
    feature_columns.remove(item)


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Build feature col
assembled_df = assembler.transform(transformed_df)

In [55]:
# Select only feature and label column
final_classify_df = assembled_df.select("features", F.col("CANCELLED").alias("label"))

In [56]:
final_classify_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



In [58]:
train, test = final_classify_df.randomSplit([.7, .3], seed=9) # 70, 30 split on balanced set or on subset of samples

In [59]:
# caching data into memory - models run quicker
train = train.repartition(32).cache()
test = test.repartition(32).cache()

### Train models

In [60]:
# Define the models
log_regress = LogisticRegression(labelCol = 'label', featuresCol = 'features')
decision_tree = DecisionTreeClassifier(labelCol = 'label', featuresCol = 'features')
rand_forest = RandomForestClassifier(labelCol = 'label', featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'label', featuresCol = 'features')

In [ ]:
log_regress_model = log_regress.fit(train)

In [62]:
decision_tree_model = decision_tree.fit(train)

In [ ]:
rand_forest_model = rand_forest.fit(train)

In [ ]:
gbt_model = gbt.fit(train)